In [7]:
import os
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [8]:
def read_parquet_by_type(train_path: str):
    # Lấy tất cả các file parquet trong thư mục
    files = [os.path.join(train_path, f) for f in os.listdir(train_path) if f.endswith('.parquet')]
    
    # Phân loại các file theo loại tên
    user_chunk_files = [file for file in files if 'user_chunk' in file]
    purchase_history_chunk_files = [file for file in files if 'purchase_history_daily_chunk' in file]
    item_chunk_files = [file for file in files if 'item_chunk' in file]
    
    # Đọc các file riêng biệt thành DataFrame
    user_chunk_df = pl.concat([pl.read_parquet(file) for file in user_chunk_files]) if user_chunk_files else None
    purchase_history_chunk_df = pl.concat([pl.read_parquet(file) for file in purchase_history_chunk_files]) if purchase_history_chunk_files else None
    item_chunk_df = pl.concat([pl.read_parquet(file) for file in item_chunk_files]) if item_chunk_files else None
    
    # Trả về một dictionary chứa các DataFrame
    return {
        "user_chunk": user_chunk_df,
        "purchase_history_chunk": purchase_history_chunk_df,
        "item_chunk": item_chunk_df
    }

In [9]:
train_path = 'D:/2025-2026/HK1-2025-2026/CS116_Python_for_ML/recommendation_dataset'
dataframes = read_parquet_by_type(train_path)

In [10]:
df_user = dataframes["user_chunk"]
df_purchase = dataframes["purchase_history_chunk"]
df_item = dataframes["item_chunk"]

# Task 4

# User chunk

| Cột              | Giữ / Loại bỏ       | Lý do                                                                                                               |
| ---------------- | ------------------- | ------------------------------------------------------------------------------------------------------------------- |
| `customer_id`, `user_id`    | ✅ Đặc trưng định danh / mã hóa| |
| `gender`         | ✅ Đặc trưng thống kê               | |
| `location`       | ✅       | |
| `province`       | ✅ Đặc trưng thống kê                |  |
| `membership`     | ✅ Đặc trưng thống kê               |   |
| `install_app`    | ✅ Đặc trưng thống kê               |  |

In [11]:
df_user_keep = df_user.select([
    pl.col("user_id"),
    pl.col("customer_id"),
    pl.col("gender"),
    pl.col("location"),
    pl.col("province"),
    pl.col("membership"),
    pl.col("install_app")
])

print("User DataFrame:")
print(df_user_keep.head(3))

User DataFrame:
shape: (3, 7)
┌───────────────────────┬─────────────┬────────┬──────────┬─────────────┬────────────┬─────────────┐
│ user_id               ┆ customer_id ┆ gender ┆ location ┆ province    ┆ membership ┆ install_app │
│ ---                   ┆ ---         ┆ ---    ┆ ---      ┆ ---         ┆ ---        ┆ ---         │
│ str                   ┆ i32         ┆ str    ┆ i32      ┆ str         ┆ str        ┆ str         │
╞═══════════════════════╪═════════════╪════════╪══════════╪═════════════╪════════════╪═════════════╡
│ e1e48206652bf8c279ff0 ┆ 14732       ┆ Nam    ┆ 155      ┆ Hồ Chí Minh ┆ Standard   ┆ In-Store    │
│ 206c69a80…            ┆             ┆        ┆          ┆             ┆            ┆             │
│ 77891759204bd27e69fb1 ┆ 15126       ┆ Nữ     ┆ 300      ┆ Hồ Chí Minh ┆ Standard   ┆ In-Store    │
│ 1a7b92889…            ┆             ┆        ┆          ┆             ┆            ┆             │
│ b8041b584a0bb66553617 ┆ 29718       ┆ Nữ     ┆ 157      ┆ B

## Xử lý Province

In [12]:
# Thay thế các giá trị "Thành Phố" và "Tỉnh" trong cột 'province'
df_user_keep = df_user_keep.with_columns(
    pl.col('province')
    .str.replace('Thành Phố ', '')  # Loại bỏ "Thành Phố"
    .str.replace('Tỉnh ', '')  # Loại bỏ "Tỉnh"
    .alias('province')  # Đặt lại tên cột sau khi thay thế
)

# In các giá trị duy nhất sau khi thay thế
unique_provinces = df_user_keep['province'].unique().to_list()
print("Unique values in column 'province' after replacement:")
print(unique_provinces)

Unique values in column 'province' after replacement:
['Trà Vinh', 'Bà Rịa - Vũng Tàu', 'Kiên Giang', 'Bến Tre', 'Bắc Kạn', 'Hậu Giang', 'Kon Tum', 'Lạng Sơn', 'Đà Nẵng', 'Bắc Giang', 'Bình Thuận', 'Hồ Chí Minh', 'Nghệ An', 'Hà Nội', 'Đắk Lắk', 'Điện Biên', 'Quảng Ngãi', 'Sơn La', 'Lào Cai', 'Hòa Bình', 'Thanh Hóa', 'Ninh Bình', 'Hà Nam', 'Quảng Trị', 'Hà Giang', 'Hải Phòng', 'Sóc Trăng', 'Bắc Ninh', 'Tiền Giang', 'An Giang', 'Yên Bái', 'Bình Định', 'Thái Bình', 'Hà Tĩnh', 'Đắk Nông', 'Gia Lai', 'Quảng Nam', 'Khánh Hòa', 'Hưng Yên', 'Hải Dương', 'Nam Định', 'Tây Ninh', 'Vĩnh Phúc', 'Ninh Thuận', 'Cần Thơ', 'Lâm Đồng', 'Long An', 'Đồng Nai', 'Vĩnh Long', 'Cao Bằng', 'Đồng Tháp', 'Thừa Thiên Huế', 'Phú Thọ', 'Bình Dương', 'Bạc Liêu', 'Thái Nguyên', 'Lai Châu', 'Cà Mau', 'Quảng Bình', 'Quảng Ninh', 'Bình Phước', 'Phú Yên', 'Tuyên Quang']


# Purchase chunk

| Cột            | Giữ / Loại bỏ | Lý do                                                                                                                                             |
| -------------- | ------------- | ------------------------------------------------------------------------------------------------------------------------------------------------- |
| `timestamp`    | ✅ Đặc trưng thời gian         |  |
| `user_id`      | ✅ Đặc trưng định danh / mã hóa        |             |
| `item_id`      | ✅ Đặc trưng định danh / mã hóa         |       |
| `price`        | ✅          |                            |
| `quantity`     | ✅ Đặc trưng thống kê         |                                                  |
| `customer_id`  | ✅ Đặc trưng định danh / mã hóa         |   |
| `created_date` | ✅ Đặc trưng thời gian      ||
| `payment`      | ✅ Đặc trưng thống kê       |                 |
| `location`     | ✅         |          |
| `discount`     | ✅   Chuẩn hóa về 0 - 1        |                                                         |
| `channel`      | ✅ Đặc trưng thống kê         |           |

In [13]:
df_purchase_keep = df_purchase.select([
    pl.col("user_id"),
    pl.col("item_id"),
    pl.col("timestamp"),
    pl.col("quantity"),
    pl.col("customer_id"),
    pl.col("created_date"),
    pl.col("payment"),
    pl.col("location"),
    pl.col("discount"),
    pl.col("channel"),
    pl.col("price")
])

print("Purchase History DataFrame:")
print(df_purchase_keep.head(3))

Purchase History DataFrame:
shape: (3, 11)
┌────────────┬────────────┬───────────┬──────────┬───┬──────────┬───────────┬──────────┬───────────┐
│ user_id    ┆ item_id    ┆ timestamp ┆ quantity ┆ … ┆ location ┆ discount  ┆ channel  ┆ price     │
│ ---        ┆ ---        ┆ ---       ┆ ---      ┆   ┆ ---      ┆ ---       ┆ ---      ┆ ---       │
│ str        ┆ str        ┆ i64       ┆ i32      ┆   ┆ i32      ┆ decimal[3 ┆ str      ┆ decimal[3 │
│            ┆            ┆           ┆          ┆   ┆          ┆ 8,4]      ┆          ┆ 8,4]      │
╞════════════╪════════════╪═══════════╪══════════╪═══╪══════════╪═══════════╪══════════╪═══════════╡
│ ca12702ddf ┆ 7115000000 ┆ 173506422 ┆ 1        ┆ … ┆ 656      ┆ 0.0000    ┆ In-Store ┆ 49000.000 │
│ 55acaa9fb7 ┆ 004        ┆ 1         ┆          ┆   ┆          ┆           ┆          ┆ 0         │
│ 67e10faaa6 ┆            ┆           ┆          ┆   ┆          ┆           ┆          ┆           │
│ …          ┆            ┆           ┆         

### Xử lý discount bằng cách chuyển sang phần trăm dựa trên price

In [14]:
import polars as pl

# Giả sử df_item là DataFrame chứa các cột 'discount' và 'price'

df_purchase_keep = df_purchase_keep.with_columns(
    # Thay thế giá trị của cột 'discount' bằng discount_percentage (giá trị từ 0 đến 1)
    (pl.col("discount") / pl.col("price")).alias("discount")
)

# Kiểm tra kết quả
print(df_purchase_keep[['price', 'discount']].head())


shape: (5, 2)
┌───────────────┬───────────────┐
│ price         ┆ discount      │
│ ---           ┆ ---           │
│ decimal[38,4] ┆ decimal[38,4] │
╞═══════════════╪═══════════════╡
│ 49000.0000    ┆ 0.0000        │
│ 69000.0000    ┆ 0.0000        │
│ 75000.0000    ┆ 0.0000        │
│ 58500.0000    ┆ 0.2222        │
│ 89000.0000    ┆ 0.1124        │
└───────────────┴───────────────┘


### Xử lý Price bằng Winsorizing

In [15]:
import polars as pl

# Giả sử df_purchase là DataFrame Polars chứa cột 'price'

# Tính IQR cho 'price'
Q1_price = df_purchase_keep['price'].quantile(0.25)
Q3_price = df_purchase_keep['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price

# Giới hạn cho Winsorizing 'price'
lower_limit_price = Q1_price - 1.5 * IQR_price
upper_limit_price = Q3_price + 1.5 * IQR_price

# Winsorizing 'price': Thay thế các giá trị ngoài giới hạn bằng giới hạn
df_purchase_keep = df_purchase_keep.with_columns(
    pl.when(pl.col('price') < lower_limit_price)
    .then(lower_limit_price)
    .otherwise(
        pl.when(pl.col('price') > upper_limit_price)
        .then(upper_limit_price)
        .otherwise(pl.col('price'))  # Giữ nguyên nếu trong phạm vi
    ).alias('price')
)

# Kiểm tra giá trị sau khi winsorizing
print("Summary statistics after Winsorizing (Price):")
print(df_purchase_keep.select('price').describe())


Summary statistics after Winsorizing (Price):
shape: (9, 2)
┌────────────┬───────────────┐
│ statistic  ┆ price         │
│ ---        ┆ ---           │
│ str        ┆ f64           │
╞════════════╪═══════════════╡
│ count      ┆ 3.5729825e7   │
│ null_count ┆ 0.0           │
│ mean       ┆ 157778.138713 │
│ std        ┆ 153952.175963 │
│ min        ┆ 0.0444        │
│ 25%        ┆ 48316.3914    │
│ 50%        ┆ 87000.0       │
│ 75%        ┆ 244000.0      │
│ max        ┆ 537525.4129   │
└────────────┴───────────────┘


# Item

| Cột                  | Giữ / Loại bỏ       | Lý do                                                                                                                                                    |
| -------------------- | ------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `created_date`       | ✅ Đặc trưng thời gian             |     |
| `creation_timestamp` | ✅ Đặc trưng thời gian               |  |
| `price`              | ✅             |  |
| `gp` (gross profit)  | ✅                | |
| `category`        | ✅ Đặc trưng quan trọng cho task 5               |                |
| `category_l1`        | ✅ Đặc trưng quan trọng cho task 5               |                |
| `gender_target`      | ✅ Đặc trưng cần xử lý |   |
| `sale_status`        | ✅                |      |
| `p_id`               | ✅ Đặc trưng định danh / mã hóa            |  |


In [16]:
df_item_keep = df_item.select([
    pl.col("p_id"), 
    pl.col("category"),
    pl.col("category_l1"),
    pl.col("created_date"),
    pl.col("creation_timestamp"),
    pl.col("price"),
    pl.col("gp"),
    pl.col("gender_target"),
    pl.col("sale_status")
])

### Xử lý Price bằng Winsorizing

In [17]:
import polars as pl

# Giả sử df_item là DataFrame Polars chứa cột 'price'

# Tính IQR cho 'price'
Q1_price = df_item_keep['price'].quantile(0.25)
Q3_price = df_item_keep['price'].quantile(0.75)
IQR_price = Q3_price - Q1_price

# Giới hạn cho Winsorizing 'price'
lower_limit_price = Q1_price - 1.5 * IQR_price
upper_limit_price = Q3_price + 1.5 * IQR_price

# Winsorizing 'price': Thay thế các giá trị ngoài giới hạn bằng giới hạn
df_item_keep = df_item_keep.with_columns(
    pl.when(pl.col('price') < lower_limit_price)
    .then(lower_limit_price)
    .otherwise(
        pl.when(pl.col('price') > upper_limit_price)
        .then(upper_limit_price)
        .otherwise(pl.col('price'))  # Giữ nguyên nếu trong phạm vi
    ).alias('price')
)

# Kiểm tra giá trị sau khi winsorizing
print("Summary statistics after Winsorizing (Price):")
print(df_item_keep.select('price').describe())


Summary statistics after Winsorizing (Price):
shape: (9, 2)
┌────────────┬───────────────┐
│ statistic  ┆ price         │
│ ---        ┆ ---           │
│ str        ┆ f64           │
╞════════════╪═══════════════╡
│ count      ┆ 27332.0       │
│ null_count ┆ 0.0           │
│ mean       ┆ 142751.84992  │
│ std        ┆ 109120.160588 │
│ min        ┆ 0.0           │
│ 25%        ┆ 49000.0       │
│ 50%        ┆ 119000.0      │
│ 75%        ┆ 199000.0      │
│ max        ┆ 424000.0      │
└────────────┴───────────────┘


### Xử lý gp bằng Winsorizing

In [18]:
import polars as pl

# Giả sử df_item là DataFrame Polars chứa cột 'price'

# Tính IQR cho 'price'
Q1_price = df_item_keep['gp'].quantile(0.25)
Q3_price = df_item_keep['gp'].quantile(0.75)
IQR_price = Q3_price - Q1_price

# Giới hạn cho Winsorizing 'price'
lower_limit_price = Q1_price - 1.5 * IQR_price
upper_limit_price = Q3_price + 1.5 * IQR_price

# Winsorizing 'price': Thay thế các giá trị ngoài giới hạn bằng giới hạn
df_item_keep = df_item_keep.with_columns(
    pl.when(pl.col('gp') < lower_limit_price)
    .then(lower_limit_price)
    .otherwise(
        pl.when(pl.col('gp') > upper_limit_price)
        .then(upper_limit_price)
        .otherwise(pl.col('gp'))  # Giữ nguyên nếu trong phạm vi
    ).alias('gp')
)

# Kiểm tra giá trị sau khi winsorizing
print("Summary statistics after Winsorizing (gp):")
print(df_item_keep.select('gp').describe())


Summary statistics after Winsorizing (gp):
shape: (9, 2)
┌────────────┬──────────────┐
│ statistic  ┆ gp           │
│ ---        ┆ ---          │
│ str        ┆ f64          │
╞════════════╪══════════════╡
│ count      ┆ 27332.0      │
│ null_count ┆ 0.0          │
│ mean       ┆ 38494.988837 │
│ std        ┆ 42347.432039 │
│ min        ┆ 0.0          │
│ 25%        ┆ 0.0          │
│ 50%        ┆ 25047.0      │
│ 75%        ┆ 65532.0      │
│ max        ┆ 163830.0     │
└────────────┴──────────────┘


### Xử lý một số giá trị "Không xác định của gender_target

In [19]:
import polars as pl

# Giả sử df_item chứa các cột 'description' và 'description_new'

# Định nghĩa các từ khóa tìm kiếm cho gender_target
gender_keywords = {
    "Bé Gái": ['bé gái', 'gái', 'nữ', 'women', 'girl'],
    "Bé Trai": ['bé trai', 'trai', 'nam', 'men', 'boy'],
    "Unisex": ['unisex'],
    "Sơ sinh": ['sơ sinh', 'newborn']
}

# Hàm để xác định giới tính (gender) trong text (description hoặc description_new)
def get_gender_from_keywords(text):
    # Kiểm tra các từ khóa trong text và trả về giới tính
    found_genders = set()
    for gender, keywords in gender_keywords.items():
        if any(keyword in text for keyword in keywords):
            found_genders.add(gender)
    if "Bé Gái" in found_genders and "Bé Trai" in found_genders:
        return "Unisex"  # Nếu tìm thấy cả "Bé Gái" và "Bé Trai", gán "Unisex"
    return found_genders.pop() if found_genders else "Không xác định"  # Nếu chỉ tìm thấy một, trả về giới tính, nếu không gán "Không xác định"


# Tạo cột mới gender_target từ description và description_new mà không dùng apply
def fill_gender_target(df):
    # Kiểm tra nếu gender_target là "Không xác định"
    condition = pl.col('gender_target') == 'Không xác định'

    # Kiểm tra nếu description chứa từ khóa giới tính và thay thế
    for gender, keywords in gender_keywords.items():
        condition_description = pl.col('description').str.contains('|'.join(keywords))
        condition_description_new = pl.col('description_new').str.contains('|'.join(keywords))
        
        # Cập nhật gender_target nếu có từ khóa trong description hoặc description_new
        df = df.with_columns(
            pl.when(condition)
            .then(
                pl.when(condition_description)
                .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description
                .otherwise(
                    pl.when(condition_description_new)
                    .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description_new
                    .otherwise(pl.lit("Không xác định"))
                )
            )
            .otherwise(pl.col('gender_target'))  # Giữ nguyên nếu không phải "Không xác định"
            .alias('gender_target')
        )
    
    return df

# Áp dụng hàm vào DataFrame
df_item = fill_gender_target(df_item)

# Kiểm tra kết quả
print(df_item[['gender_target', 'description', 'description_new']].head())


shape: (5, 3)
┌────────────────┬─────────────────────────────────┬─────────────────────────────────┐
│ gender_target  ┆ description                     ┆ description_new                 │
│ ---            ┆ ---                             ┆ ---                             │
│ str            ┆ str                             ┆ str                             │
╞════════════════╪═════════════════════════════════╪═════════════════════════════════╡
│ Không xác định ┆ Không xác định                  ┆ Chi tiết sản phẩm             … │
│ Bé Gái         ┆ Không xác định                  ┆ Không xác định                  │
│ Không xác định ┆ - Chất liệu: Sản phẩm được làm… ┆ Chi tiết sản phẩm             … │
│ Không xác định ┆ ﻿﻿Tã dán Merries size S 82 miế…   ┆ Không xác định                  │
│ Không xác định ┆ ﻿﻿﻿Bỉm tã quần Merries size M …    ┆ Không xác định                  │
└────────────────┴─────────────────────────────────┴─────────────────────────────────┘


In [20]:
print(df_item['gender_target'].value_counts())

shape: (5, 2)
┌────────────────┬───────┐
│ gender_target  ┆ count │
│ ---            ┆ ---   │
│ str            ┆ u32   │
╞════════════════╪═══════╡
│ Unisex         ┆ 6     │
│ Sơ sinh        ┆ 3063  │
│ Không xác định ┆ 14880 │
│ Bé Trai        ┆ 4319  │
│ Bé Gái         ┆ 5064  │
└────────────────┴───────┘


In [21]:
import polars as pl

# Giả sử df_item chứa các cột 'description' và 'description_new'

# Định nghĩa các từ khóa tìm kiếm cho gender_target
gender_keywords = {
    "Bé Gái": ['bé gái', 'gái', 'nữ', 'women', 'girl'],
    "Bé Trai": ['bé trai', 'trai', 'nam', 'men', 'boy'],
    "Unisex": ['unisex','thời trang' ],
    "Sơ sinh": ['sơ sinh', 'newborn', 'tã', 'babycare', 'hóa mỹ phẩm cho bé']
}

# Hàm để xác định giới tính (gender) trong text (description hoặc description_new)
def get_gender_from_keywords(text):
    # Kiểm tra các từ khóa trong text và trả về giới tính
    found_genders = set()
    for gender, keywords in gender_keywords.items():
        if any(keyword in text for keyword in keywords):
            found_genders.add(gender)
    if "Bé Gái" in found_genders and "Bé Trai" in found_genders:
        return "Unisex"  # Nếu tìm thấy cả "Bé Gái" và "Bé Trai", gán "Unisex"
    return found_genders.pop() if found_genders else "Không xác định"  # Nếu chỉ tìm thấy một, trả về giới tính, nếu không gán "Không xác định"


# Tạo cột mới gender_target từ description và description_new mà không dùng apply
def fill_gender_target(df):
    # Kiểm tra nếu gender_target là "Không xác định"
    condition = pl.col('gender_target') == 'Không xác định'

    # Kiểm tra nếu description chứa từ khóa giới tính và thay thế
    for gender, keywords in gender_keywords.items():
        condition_description = pl.col('category').str.contains('|'.join(keywords))
        condition_description_new = pl.col('category_l1').str.contains('|'.join(keywords))
        
        # Cập nhật gender_target nếu có từ khóa trong description hoặc description_new
        df = df.with_columns(
            pl.when(condition)
            .then(
                pl.when(condition_description)
                .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description
                .otherwise(
                    pl.when(condition_description_new)
                    .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description_new
                    .otherwise(pl.lit("Không xác định"))
                )
            )
            .otherwise(pl.col('gender_target'))  # Giữ nguyên nếu không phải "Không xác định"
            .alias('gender_target')
        )
    
    return df

# Áp dụng hàm vào DataFrame
df_item = fill_gender_target(df_item)

# Kiểm tra kết quả
print(df_item[['gender_target', 'category', 'category_l1']].head())


shape: (5, 3)
┌────────────────┬───────────────────┬────────────────┐
│ gender_target  ┆ category          ┆ category_l1    │
│ ---            ┆ ---               ┆ ---            │
│ str            ┆ str               ┆ str            │
╞════════════════╪═══════════════════╪════════════════╡
│ Không xác định ┆ Núm ty Dr Brown   ┆ Babycare       │
│ Bé Gái         ┆ Bộ quần áo bé gái ┆ Thời trang     │
│ Không xác định ┆ Gặm nướu khác     ┆ Đồ chơi & Sách │
│ Không xác định ┆ Merries_Sơ Sinh   ┆ Tã             │
│ Không xác định ┆ Merries_Tã Quần   ┆ Tã             │
└────────────────┴───────────────────┴────────────────┘


In [22]:
print(df_item['gender_target'].value_counts())

shape: (5, 2)
┌────────────────┬───────┐
│ gender_target  ┆ count │
│ ---            ┆ ---   │
│ str            ┆ u32   │
╞════════════════╪═══════╡
│ Unisex         ┆ 6     │
│ Bé Trai        ┆ 5410  │
│ Không xác định ┆ 10703 │
│ Bé Gái         ┆ 6422  │
│ Sơ sinh        ┆ 4791  │
└────────────────┴───────┘


In [23]:
import polars as pl

# Giả sử df_item chứa các cột 'description' và 'description_new'

# Định nghĩa các từ khóa tìm kiếm cho gender_target
gender_keywords = {
    "Bé Gái": ['bé gái', 'gái', 'nữ', 'women', 'girl'],
    "Bé Trai": ['bé trai', 'trai', 'nam', 'men', 'boy'],
    "Unisex": ['unisex'],
    "Sơ sinh": ['sơ sinh', 'newborn']
}

# Hàm để xác định giới tính (gender) trong text (description hoặc description_new)
def get_gender_from_keywords(text):
    # Kiểm tra các từ khóa trong text và trả về giới tính
    found_genders = set()
    for gender, keywords in gender_keywords.items():
        if any(keyword in text for keyword in keywords):
            found_genders.add(gender)
    if "Bé Gái" in found_genders and "Bé Trai" in found_genders:
        return "Unisex"  # Nếu tìm thấy cả "Bé Gái" và "Bé Trai", gán "Unisex"
    return found_genders.pop() if found_genders else "Không xác định"  # Nếu chỉ tìm thấy một, trả về giới tính, nếu không gán "Không xác định"


# Tạo cột mới gender_target từ description và description_new mà không dùng apply
def fill_gender_target(df):
    # Kiểm tra nếu gender_target là "Không xác định"
    condition = pl.col('gender_target') == 'Không xác định'

    # Kiểm tra nếu description chứa từ khóa giới tính và thay thế
    for gender, keywords in gender_keywords.items():
        condition_description = pl.col('category_l2').str.contains('|'.join(keywords))
        condition_description_new = pl.col('category_l3').str.contains('|'.join(keywords))
        
        # Cập nhật gender_target nếu có từ khóa trong description hoặc description_new
        df = df.with_columns(
            pl.when(condition)
            .then(
                pl.when(condition_description)
                .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description
                .otherwise(
                    pl.when(condition_description_new)
                    .then(pl.lit(gender))  # Nếu tìm thấy từ khóa trong description_new
                    .otherwise(pl.lit("Không xác định"))
                )
            )
            .otherwise(pl.col('gender_target'))  # Giữ nguyên nếu không phải "Không xác định"
            .alias('gender_target')
        )
    
    return df

# Áp dụng hàm vào DataFrame
df_item = fill_gender_target(df_item)

# Kiểm tra kết quả
print(df_item[['gender_target', 'category_l2', 'category_l3']].head())


shape: (5, 3)
┌────────────────┬────────────────────┬───────────────────────────────┐
│ gender_target  ┆ category_l2        ┆ category_l3                   │
│ ---            ┆ ---                ┆ ---                           │
│ str            ┆ str                ┆ str                           │
╞════════════════╪════════════════════╪═══════════════════════════════╡
│ Không xác định ┆ Bình sữa, phụ kiện ┆ Núm ty                        │
│ Bé Gái         ┆ Cơ cấu hàng cũ     ┆ Thời trang bé trai, bé gái cũ │
│ Không xác định ┆ 0-1Y               ┆ Gặm nướu                      │
│ Không xác định ┆ Merries            ┆ Merries                       │
│ Không xác định ┆ Merries            ┆ Merries                       │
└────────────────┴────────────────────┴───────────────────────────────┘


In [24]:
print(df_item['gender_target'].value_counts())

shape: (5, 2)
┌────────────────┬───────┐
│ gender_target  ┆ count │
│ ---            ┆ ---   │
│ str            ┆ u32   │
╞════════════════╪═══════╡
│ Bé Trai        ┆ 5446  │
│ Sơ sinh        ┆ 5257  │
│ Bé Gái         ┆ 7075  │
│ Không xác định ┆ 9548  │
│ Unisex         ┆ 6     │
└────────────────┴───────┘


In [ ]:
print(df_item['category_l1'].unique().to_list())

['Textile', 'Thời trang', 'Vệ sinh', 'Sữa nước', 'Hóa mỹ phẩm cho bé', 'Phụ kiện', 'Thực phẩm cho gia đình', 'Thực phẩm cho bé', 'Đồ chơi & Sách', 'Tã', 'Hóa mỹ phẩm gia đình', 'Babycare', 'TPCN', 'Gói Hội Viên', 'Sữa']


In [26]:
print(df_item['category_l2'].unique().to_list())

['Băng vệ sinh', 'Abbott', 'Kẹo', 'Vệ sinh cho bé', 'Pampers', 'Chăm sóc sức khỏe bé', 'Giặt xả cho bé', 'Bubs', 'Sữa tươi tiệt trùng', 'Morinaga', 'Sức khỏe gia đình', 'TP từ sữa (bảo quản lạnh)', 'Khăn em bé', 'Cơ cấu hàng tồn', 'Mì & Đồ khô ăn liền', 'Meiji', 'Quần áo & Phụ kiện sơ sinh', 'Gối', 'Kid Essential', 'Đồ dùng cho mẹ', 'Nestle', 'Thời trang bé gái', 'Bánh', 'Sữa cho mẹ', 'Takato', 'Phụ kiện khác', 'Sữa có vị', 'Giấy', 'Bông tẩy trang', 'Chăm sóc răng miệng', 'Chăm sóc mẹ trước & sau sinh', 'Hipp', 'Merries', 'TP bảo quản lạnh', 'Nutricare', 'Bellamy', 'Sữa bột pha sẵn', 'Humana', 'Kế hoạch gia đình', 'Moony', 'Gói Hội Viên BẦU', 'Khăn ướt', 'Vitadairy', 'TPCN cho gia đình', 'Nón', 'A2 milk company', 'Vệ sinh dụng cụ & bề mặt', 'Nutifood', 'Meiji Nội địa', 'Bông gạc', '0-1Y', 'Chăm sóc da', 'Giày tập đi', 'Thời trang bé trai', 'Bé ngủ', 'Chăm sóc da bé', 'Chăm sóc tóc', 'Snack', 'Dad and Me', 'Chăn', 'SP mùa vụ', 'Mứt & Bơ phết', 'Bobby', 'Nước rửa bình sữa', 'Sweety', 'Ge

In [27]:
print(df_item['category_l3'].unique().to_list())

['Enfa', 'Bubs Organic', 'Máy rửa bình sữa', 'Vinamilk', 'Viên sủi Vitamin', 'Khoai tây chiên', 'Enfa US', 'Máy xay', 'Sandal bé trai', 'Phụ kiện cho mẹ', 'FCV', 'Bánh mì', 'Phô mai sấy', 'Bobby Quần Mở Một Bên', 'Xúc xích', 'Nui & Mì', 'Bánh quế', 'Nước trái cây', 'Vận động trong nhà', 'Khăn choàng ủ', 'Khăn ướt gia dụng', 'Dung dịch vệ sinh cho bé', 'Kiwi', 'Ngủ ngon', 'Maeil', 'Trà', 'Phụ  kiện tồn', 'Nước có thạch', 'A2 milk company', 'Bánh Bolo', 'Thiết bị y tế', 'Trữ sữa', 'Sữa ong chúa', 'Grow Plus trắng', 'Kabrita', 'Kẹo dẻo', 'Height Boosting', 'Gummies', 'Nón tồn', 'Giặt gia đình', 'Phấn rôm & phấn thơm', 'NAN Supreme', 'Pin', 'Tương & Sốt', 'Pampers Overnight', 'Phụ kiện khác', 'Anlene', 'Quần', 'Nhập vai', 'Khẩu trang người lớn', 'Nước rửa đa năng', 'Bàn chải', 'Nutifood', 'Kid Essential', 'Bột rau', 'Sách', 'Huggies Thường', 'Dưỡng tóc', 'Nón sơ sinh', 'Sữa tươi thanh trùng', 'Vitamin D', 'Takato', 'Chăn vải chần bông', 'Nestle', 'PreNAN', 'Sắt', 'Khăn giấy bỏ túi', 'Gạc, 